In [1]:
import pymn
import pandas as pd
import matplotlib.pyplot as plt
import scanpy as sc
import anndata as ad
import numpy as np
import itertools
import scipy.stats
import seaborn as sns
import scgen 
sc.settings.verbosity = 3   
sc.set_figure_params(figsize = (15,15),facecolor = 'white')

Global seed set to 0
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/home/passala/miniconda3/envs/Single_cell_data_fix/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [6]:
alias_File = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/zma_gene_aliases.txt', sep = '\t',)
alias_File['symbol'] = alias_File['symbol'].fillna(alias_File['locus_name'])
alias_File

,locus_name,symbol,full_name
0,Zm00001d027231,LOC100382519,C0P8I2
1,Zm00001d027259,Zm00001d027259,A0A1D6JJF9
2,Zm00001d027267,LOC100193420,B4FEZ2
3,Zm00001d027268,LOC100283177,B4FS29
4,Zm00001d027276,LOC103630457,A0A1D6JJP0
...,...,...,...
46267,ENSRNA049478543,ENSRNA049478543,NaN
46268,ENSRNA049478539,ENSRNA049478539,NaN
46269,ENSRNA049478900,ENSRNA049478900,NaN
46270,ENSRNA049478881,ENSRNA049478881,NaN


In [ ]:
alias_File.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/filled_zma_gene_aliases.txt', index= False, sep = '\t')

In [2]:
maize_table = pd.read_csv('/data/CoCoCoNet/geneInfo/maize_info.csv')
maize_table = maize_table[['EnsemblID','GeneSymbol','UniProtID']]
maize_table.columns = ['locus_name','symbol','full_name']
maize_table.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/data_zma/zma_gene_aliases.txt', index= False, sep = '\t')

In [3]:
tf_list = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Initialization/zma_TF2fam2mot.txt', sep = '\t')
tf_list = tf_list['Zm00001d050368']
tf_list

0       Zm00001d037159
1       Zm00001d037425
2       Zm00001d020951
3       Zm00001d032239
4       Zm00001d051861
             ...      
2142    Zm00001d053687
2143    Zm00001d048105
2144    Zm00001d039118
2145    Zm00001d003284
2146    Zm00001d000110
Name: Zm00001d050368, Length: 2147, dtype: object

In [4]:
#tf_list.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/TF_list.txt', index= False, sep = '\t', header = False)

In [5]:
norm_data = sc.read_h5ad('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/faster_reading_full_data.h5ad')

In [6]:
cell_identity = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/Meta_Maize_Sorghum_Setaria_Nuclei_Cells_SCT_OrthologousgenesGoodname_Sept2022.csv',index_col = 0)
norm_data.obs = cell_identity

In [7]:
norm_data.obs = norm_data.obs.rename(columns = {'orig.ident':'Batch','Annotation':'Cell Cluster Annotation','type':'Type'})

In [8]:
norm_data.var['ZM'] = norm_data.var_names.str.startswith('Zm')
norm_data = norm_data[:, norm_data.var['ZM'] == True] 

In [9]:
cell_type_list = norm_data.obs['Cell Cluster Annotation'].unique()

In [10]:
fake_cluster_id = np.arange(len(cell_type_list))
fake_cluster_id

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [11]:
cluster_2_ident = pd.DataFrame(data = zip(fake_cluster_id,cell_type_list))
cluster_2_ident

,0,1
0,0,Cortex_4
1,1,Stele_2
2,2,Cortex_1
3,3,Epidermis_2
4,4,Young_Stele_2
5,5,Stele_1
6,6,Epidermis_1
7,7,Cortex_3
8,8,Pericycle
9,9,Epidermis_3


In [12]:
norm_data.obs['Cell ID']= norm_data.obs.index
norm_data.obs

/tmp/ipykernel_3965551/3738641279.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  norm_data.obs['Cell ID']= norm_data.obs.index


,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID
Maize_Cell1_AAACCTGAGAGCTTCT,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGAGAGCTTCT
Maize_Cell1_AAACCTGAGTGATCGG,Maize_Cell_1,11556,3579,14008,3580,7,7,Maize,Stele_2,Cell,Maize_Cell1_AAACCTGAGTGATCGG
Maize_Cell1_AAACCTGCACTGTCGG,Maize_Cell_1,19725,4690,15803,4688,1,1,Maize,Cortex_1,Cell,Maize_Cell1_AAACCTGCACTGTCGG
Maize_Cell1_AAACCTGGTGTGAAAT,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGGTGTGAAAT
Maize_Cell1_AAACCTGTCTCAAGTG,Maize_Cell_1,11246,3548,13956,3546,14,14,Maize,Epidermis_2,Cell,Maize_Cell1_AAACCTGTCTCAAGTG
...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3,9192,2954,21040,3212,14,14,Setaria,Epidermis_2,Cell,Set_Cells_3_TTTGGTTTCTGGGTCG
Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3,27048,4655,23549,4654,16,16,Setaria,Trichoblast,Cell,Set_Cells_3_TTTGTTGAGGCTCACC
Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3,21855,5009,22238,5009,17,17,Setaria,Phloem,Cell,Set_Cells_3_TTTGTTGCAACCAGAG
Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3,9015,2669,20346,3006,3,3,Setaria,LRC,Cell,Set_Cells_3_TTTGTTGCAGTCTACA


In [28]:
maize_cells= norm_data[norm_data.obs['Species'] == 'Maize', :]
sorghum_cells= norm_data[norm_data.obs['Species'] == 'Sorghum', :]
setaria_cells= norm_data[norm_data.obs['Species'] == 'Setaria', :]

In [88]:
maize_cells

View of AnnData object with n_obs × n_vars = 7615 × 29539
    obs: 'Batch', 'nCount_RNA', 'nFeature_RNA', 'nCount_SCT', 'nFeature_SCT', 'integrated_snn_res.0.6', 'seurat_clusters', 'Species', 'Cell Cluster Annotation', 'Type', 'Cell ID'
    var: 'ZM'

In [71]:
cell_2_clusters = setaria_cells.obs.merge(left_on='Cell Cluster Annotation',right = cluster_2_ident, right_on= 1)

In [72]:
cell_2_clusters

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID,0,1
0,Set_Cells,2524,1465,3060,1459,2,2,Setaria,Cortex_2,Cell,Set_Cell_AAACCTGAGATGGCGT-1,11,Cortex_2
1,Set_Cells,2284,1135,3178,1133,2,2,Setaria,Cortex_2,Cell,Set_Cell_CACAAACTCATAACCG-1,11,Cortex_2
2,Set_Cells,2430,1200,3205,1199,2,2,Setaria,Cortex_2,Cell,Set_Cell_CACAAACTCATGGTCA-1,11,Cortex_2
3,Set_Cells,3357,1505,3403,1497,2,2,Setaria,Cortex_2,Cell,Set_Cell_CCCAGTTAGTTCGATC-1,11,Cortex_2
4,Set_Cells,2047,1136,3053,1136,2,2,Setaria,Cortex_2,Cell,Set_Cell_CCCTCCTGTCGCATCG-1,11,Cortex_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26617,Set_Cells_3,24477,5850,23310,5849,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGGATGTCTATCGCC,4,Young_Stele_2
26618,Set_Cells_3,53617,7975,23469,6008,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGGGCGGTAGCGTAG,4,Young_Stele_2
26619,Set_Cells_3,58743,8175,23242,5776,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGTTTGTCGCTGTCT,4,Young_Stele_2
26620,Set_Cells_3,51386,7763,23434,5922,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTTATGCGTCGTATTG,4,Young_Stele_2


In [73]:
cell_2_clusters = cell_2_clusters.rename(columns = {0:'Forced Clusters'})
cell_2_clusters

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID,Forced Clusters,1
0,Set_Cells,2524,1465,3060,1459,2,2,Setaria,Cortex_2,Cell,Set_Cell_AAACCTGAGATGGCGT-1,11,Cortex_2
1,Set_Cells,2284,1135,3178,1133,2,2,Setaria,Cortex_2,Cell,Set_Cell_CACAAACTCATAACCG-1,11,Cortex_2
2,Set_Cells,2430,1200,3205,1199,2,2,Setaria,Cortex_2,Cell,Set_Cell_CACAAACTCATGGTCA-1,11,Cortex_2
3,Set_Cells,3357,1505,3403,1497,2,2,Setaria,Cortex_2,Cell,Set_Cell_CCCAGTTAGTTCGATC-1,11,Cortex_2
4,Set_Cells,2047,1136,3053,1136,2,2,Setaria,Cortex_2,Cell,Set_Cell_CCCTCCTGTCGCATCG-1,11,Cortex_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26617,Set_Cells_3,24477,5850,23310,5849,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGGATGTCTATCGCC,4,Young_Stele_2
26618,Set_Cells_3,53617,7975,23469,6008,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGGGCGGTAGCGTAG,4,Young_Stele_2
26619,Set_Cells_3,58743,8175,23242,5776,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTGTTTGTCGCTGTCT,4,Young_Stele_2
26620,Set_Cells_3,51386,7763,23434,5922,9,9,Setaria,Young_Stele_2,Cell,Set_Cells_3_TTTATGCGTCGTATTG,4,Young_Stele_2


In [74]:
cell_2_clusters = cell_2_clusters[['Cell ID','Forced Clusters','Cell Cluster Annotation']]

In [75]:
cell_2_clusters

,Cell ID,Forced Clusters,Cell Cluster Annotation
0,Set_Cell_AAACCTGAGATGGCGT-1,11,Cortex_2
1,Set_Cell_CACAAACTCATAACCG-1,11,Cortex_2
2,Set_Cell_CACAAACTCATGGTCA-1,11,Cortex_2
3,Set_Cell_CCCAGTTAGTTCGATC-1,11,Cortex_2
4,Set_Cell_CCCTCCTGTCGCATCG-1,11,Cortex_2
...,...,...,...
26617,Set_Cells_3_TTGGATGTCTATCGCC,4,Young_Stele_2
26618,Set_Cells_3_TTGGGCGGTAGCGTAG,4,Young_Stele_2
26619,Set_Cells_3_TTGTTTGTCGCTGTCT,4,Young_Stele_2
26620,Set_Cells_3_TTTATGCGTCGTATTG,4,Young_Stele_2


In [76]:
cell_2_clusters.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/Setaria_only_input/setaria_only_cells2clusters.txt', index= False, sep = '\t', header = False)

In [19]:
norm_data.obs

,Batch,nCount_RNA,nFeature_RNA,nCount_SCT,nFeature_SCT,integrated_snn_res.0.6,seurat_clusters,Species,Cell Cluster Annotation,Type,Cell ID
Maize_Cell1_AAACCTGAGAGCTTCT,Maize_Cell_1,7907,3206,13445,3280,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGAGAGCTTCT
Maize_Cell1_AAACCTGAGTGATCGG,Maize_Cell_1,11556,3579,14008,3580,7,7,Maize,Stele_2,Cell,Maize_Cell1_AAACCTGAGTGATCGG
Maize_Cell1_AAACCTGCACTGTCGG,Maize_Cell_1,19725,4690,15803,4688,1,1,Maize,Cortex_1,Cell,Maize_Cell1_AAACCTGCACTGTCGG
Maize_Cell1_AAACCTGGTGTGAAAT,Maize_Cell_1,17272,4355,15469,4354,6,6,Maize,Cortex_4,Cell,Maize_Cell1_AAACCTGGTGTGAAAT
Maize_Cell1_AAACCTGTCTCAAGTG,Maize_Cell_1,11246,3548,13956,3546,14,14,Maize,Epidermis_2,Cell,Maize_Cell1_AAACCTGTCTCAAGTG
...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3,9192,2954,21040,3212,14,14,Setaria,Epidermis_2,Cell,Set_Cells_3_TTTGGTTTCTGGGTCG
Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3,27048,4655,23549,4654,16,16,Setaria,Trichoblast,Cell,Set_Cells_3_TTTGTTGAGGCTCACC
Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3,21855,5009,22238,5009,17,17,Setaria,Phloem,Cell,Set_Cells_3_TTTGTTGCAACCAGAG
Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3,9015,2669,20346,3006,3,3,Setaria,LRC,Cell,Set_Cells_3_TTTGTTGCAGTCTACA


In [20]:
#cluster_2_ident.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/INPUTS/forced_cluster_identities.txt', index= False, sep = '\t')

In [77]:
panda_of_expression_results = pd.DataFrame(index = setaria_cells.obs.index, columns= setaria_cells.var.index, data = setaria_cells.X )
panda_of_expression_results

,Zm00001d027230,Zm00001d027231,Zm00001d027236,Zm00001d027239,Zm00001d027240,Zm00001d027242,Zm00001d027250,Zm00001d027256,Zm00001d027257,Zm00001d027258,...,Zm00001d002687,Zm00001d052418,Zm00001d052397,Zm00001d053127,Zm00001d033915,Zm00001d033266,Zm00001d032469,Zm00001d029568,Zm00001d047419,Zm00001d028222
Set_Cell_AAACCTGAGATGGCGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACCTGGTCGTGGCT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGAGAGTACAT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGCATAAAGGT-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGCATGTCTCC-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGAGGCTCACC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAACCAGAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAGTCTACA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [78]:
filtered_non_expressed_genes = panda_of_expression_results.loc[:,(panda_of_expression_results.sum()>1)]



In [79]:
filtered_non_expressed_genes

,Zm00001d027257,Zm00001d027259,Zm00001d027265,Zm00001d027266,Zm00001d027267,Zm00001d027268,Zm00001d027276,Zm00001d027277,Zm00001d027281,Zm00001d027283,...,Zm00001d027490,Zm00001d006416,Zm00001d037737,Zm00001d038726,Zm00001d023869,Zm00001d035907,Zm00001d036242,Zm00001d002772,Zm00001d033915,Zm00001d029568
Set_Cell_AAACCTGAGATGGCGT-1,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACCTGGTCGTGGCT-1,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGAGAGTACAT-1,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGCATAAAGGT-1,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cell_AAACGGGCATGTCTCC-1,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Set_Cells_3_TTTGGTTTCTGGGTCG,0.0,0.73616,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGAGGCTCACC,0.0,0.00000,0.0,0.0,0.553555,0.0,0.314601,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAACCAGAG,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,1.718402,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Set_Cells_3_TTTGTTGCAGTCTACA,0.0,0.00000,0.0,0.0,0.000000,0.0,0.746338,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
transposed_matrix = filtered_non_expressed_genes.T
transposed_matrix.index.name = 'geneid'

In [81]:
transposed_matrix

,Set_Cell_AAACCTGAGATGGCGT-1,Set_Cell_AAACCTGGTCGTGGCT-1,Set_Cell_AAACGGGAGAGTACAT-1,Set_Cell_AAACGGGCATAAAGGT-1,Set_Cell_AAACGGGCATGTCTCC-1,Set_Cell_AAACGGGGTATATGGA-1,Set_Cell_AAAGCAAAGAACTCGG-1,Set_Cell_AAAGCAACAAAGCAAT-1,Set_Cell_AAAGCAAGTCACAAGG-1,Set_Cell_AAAGCAATCGAATGGG-1,...,Set_Cells_3_TTTGGAGGTTCGGCTG,Set_Cells_3_TTTGGAGTCTGTAACG,Set_Cells_3_TTTGGTTAGCCGATCC,Set_Cells_3_TTTGGTTAGTCACTGT,Set_Cells_3_TTTGGTTGTTACCCAA,Set_Cells_3_TTTGGTTTCTGGGTCG,Set_Cells_3_TTTGTTGAGGCTCACC,Set_Cells_3_TTTGTTGCAACCAGAG,Set_Cells_3_TTTGTTGCAGTCTACA,Set_Cells_3_TTTGTTGGTGGTCTCG
geneid,,,,,,,,,,,,,,,,,,,,,
Zm00001d027257,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.299529,0.00000,0.000000,0.0,0.0,0.000000
Zm00001d027259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.529714,0.73616,0.000000,0.0,0.0,0.000000
Zm00001d027265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000
Zm00001d027266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000
Zm00001d027267,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.245026,0.0,0.0,0.0,0.000000,0.00000,0.553555,0.0,0.0,0.648407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zm00001d035907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000
Zm00001d036242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000
Zm00001d002772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.0,0.0,0.000000


In [82]:
transposed_matrix.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/Setaria_only_input/setaria_only_matrix.txt', sep = '\t')

In [83]:
csv_version_markers = pd.read_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Post_review_Dataset/setaria_high_purity_markers.csv', index_col= 0)
csv_version_markers

,X,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
171,Zm00001d020392,1.059261e-148,0.265173,0.206,0.062,6.430560e-144,Stele_1,Zm00001d020392
229,Zm00001d046596,4.776466e-114,0.277340,0.178,0.059,2.899697e-109,Stele_1,Zm00001d046596
560,Zm00001d027924,2.684380e-44,0.309600,0.175,0.092,1.629633e-39,Stele_1,Zm00001d027924
629,Zm00001d028486,0.000000e+00,1.209843,0.342,0.068,0.000000e+00,Cortex_1,Zm00001d028486
633,Zm00001d048513,0.000000e+00,0.771999,0.344,0.077,0.000000e+00,Cortex_1,Zm00001d048513
...,...,...,...,...,...,...,...,...
15692,Zm00001d0033654,5.906458e-65,0.282255,0.320,0.092,3.585693e-60,Epidermis_3,Zm00001d003365
15711,Zm00001d0284472,9.193392e-60,0.259416,0.304,0.090,5.581124e-55,Epidermis_3,Zm00001d028447
15792,Zm00001d0365741,8.364930e-38,0.327720,0.269,0.094,5.078182e-33,Epidermis_3,Zm00001d036574
15807,Zm00001d0243782,3.089264e-35,0.413370,0.167,0.047,1.875431e-30,Epidermis_3,Zm00001d024378


In [84]:
csv_version_markers = csv_version_markers.merge(left_on='cluster',right = cluster_2_ident, right_on= 1)
csv_version_markers = csv_version_markers.drop(columns= ['cluster',1])
csv_version_markers
csv_version_markers = csv_version_markers.rename(columns = {0:'cluster', 'X':'geneID'})
csv_version_markers
csv_version_markers = csv_version_markers[['geneID','p_val','avg_log2FC','pct.1','pct.2','p_val_adj','cluster','gene']]
csv_version_markers

,geneID,p_val,avg_log2FC,pct.1,pct.2,p_val_adj,cluster,gene
0,Zm00001d020392,1.059261e-148,0.265173,0.206,0.062,6.430560e-144,5,Zm00001d020392
1,Zm00001d046596,4.776466e-114,0.277340,0.178,0.059,2.899697e-109,5,Zm00001d046596
2,Zm00001d027924,2.684380e-44,0.309600,0.175,0.092,1.629633e-39,5,Zm00001d027924
3,Zm00001d028486,0.000000e+00,1.209843,0.342,0.068,0.000000e+00,2,Zm00001d028486
4,Zm00001d048513,0.000000e+00,0.771999,0.344,0.077,0.000000e+00,2,Zm00001d048513
...,...,...,...,...,...,...,...,...
1718,Zm00001d0033654,5.906458e-65,0.282255,0.320,0.092,3.585693e-60,9,Zm00001d003365
1719,Zm00001d0284472,9.193392e-60,0.259416,0.304,0.090,5.581124e-55,9,Zm00001d028447
1720,Zm00001d0365741,8.364930e-38,0.327720,0.269,0.094,5.078182e-33,9,Zm00001d036574
1721,Zm00001d0243782,3.089264e-35,0.413370,0.167,0.047,1.875431e-30,9,Zm00001d024378


In [85]:
csv_version_markers.to_csv('/data/passala/Collaborator_Data/Bruno_Ken_Collab/Mini_EX_Files/Full_clone/MINI-EX/Setaria_only_input/setaria_only_allMarkers.txt', index= False, sep = '\t')